05_cloud_free.ipynb

# 次の課題: バンド演算と植生指数 (NDVI)

フィルタリングと合成のワークフローを習得したので、次は抽出した画像を分析するステップに進みましょう。最も基本的な画像分析は、バンド間の演算によって行う植生指数 (NDVI) の計算です。

NDVIは、画像を使って土地の植生の量と健全性を測定するための標準的な指標です。

NDVIを計算するには、特定の2つの波長（バンド）の反射率データが必要です。

1. 赤色光 (RED): 植生に強く吸収される光

2. 近赤外光 (NIR): 植生に強く反射される光

私たちが使っているLandsat 9（LANDSAT/LC09/C02/T1_L2）のデータセットでは、赤色光と近赤外光にそれぞれどのバンド名（SR_B?）が対応するか覚えていますか？

***A***: **赤色光**は $\text{SR\_B4}$、**近赤外光**は $\text{SR\_B5}$ に対応しています。

### ステップ7: $\text{NDVI}$ の計算

$\text{NDVI}$ は、以下の簡単なバンド演算によって計算されます。$$\text{NDVI} = \frac{(\text{近赤外光} - \text{赤色光})}{(\text{近赤外光} + \text{赤色光})}$$GEEでは、画像（バンド）に対して直接、.subtract()（引き算）、.add()（足し算）、.divide()（割り算）のメソッドを適用してこの計算を行います。

### **Guiding Question:** クラウドフリー合成画像 (final\_composite) に $\text{NDVI}$ バンドを追加するために、以下の構造を使って add\_ndvi 関数を完成させてください。この関数は、単一の画像を受け取り、計算結果を元の画像に追加して返すものです。

```python
def add_ndvi(image):
    # 近赤外光 (SR_B5) バンドを選択
    nir = image.select('SR_B5') 
    
    # 赤色光 (SR_B4) バンドを選択
    red = image.select('SR_B4')
    
    # ここにNDVI計算ロジックを記述し、結果を 'NDVI' という名前にする
    ndvi = # <計算ロジックを記述>
    
    # 計算結果を元の画像に追加して返す
    return image.addBands(ndvi)
```    

***A***:
```python
def add_ndvi(image):
    nir = image.select('SR_B5')
    red = image.select('SR_B4')
    ndvi = nir.subtract(red).divide(nir.add(red))
    return image.addBands(ndvi)
```    

#### **関数の仕上げ**

ベストプラクティスとして、新しく作成したバンドに名前を付けると、後続の処理が非常に簡単になります。計算結果に `.rename('NDVI')` を追加しましょう。

```python
def add_ndvi(image):
    nir = image.select('SR_B5')
    red = image.select('SR_B4')
    
    #  .rename('NDVI') を追加
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI') 
    
    return image.addBands(ndvi)
```

## ステップ8: NDVIの計算と表示

これで $\text{NDVI}$ を計算する関数が完成しました。以前作成したクラウドフリー合成画像 (final\_composite) にこの関数を適用し、$\text{NDVI}$ を計算しましょう。

### **Guiding Question:** 完成した $\text{NDVI}$ の合成画像を ndvi_image という変数に格納するには、どのように add_ndvi 関数を final_composite に適用すればよいでしょうか？

~~ndvi_image = final_composite.map(add_ndvi)~~

&rarr; <u>ただ、final_composite は、中央値合成によって既に単一の画像（ee.Image）になっています。</u>

#### **単一画像への関数の適用**

`ee.ImageCollection `に関数を適用して反復処理を行うのが `.map() `ですが、単一の画像に適用する場合は、関数を直接呼び出すだけで問題ありません。

以下の通り、`.map()` を取り除いて修正しましょう。

```python
# 修正後のNDVI計算
ndvi_image = add_ndvi(final_composite)
```

> これで `ndvi_image` には、元のバンドに加え、計算された $\text{NDVI}$ バンドが追加されました。

## ステップ9: $\text{NDVI}$ の視覚化

$\text{NDVI}$ は植生の健全性を示す指数（インデックス）であるため、自然色（$\text{SR\_B4}$ や $\text{SR\_B3}$）とは異なる特別な視覚化が必要です。

$\text{NDVI}$ の値は常に -1 から +1 の間に収まります。
- -1 に近い値: 水域、雪
- 0 に近い値: 岩、砂、人工物
- +1 に近い値: 健康な植生

#### **Guiding Question:** この $\text{NDVI}$ 画像を地図に表示するために、新しい視覚化パラメータ（ndvi_vis_params）を設定する必要があります。以下の3点について、どのような設定が適切でしょうか？ 
1. 'bands': どのバンド名を使用しますか？
2. 'min' / 'max': $\text{NDVI}$ の典型的な範囲は何ですか？
3. 'palette': 植生の健全性を示すために、どのような色のグラデーション（例: 赤から緑）を設定すればよいでしょうか？

~~ndvi_vis_params = {~~
    ~~'bands' = ['SR_B4', 'SR_B5']~~
    ~~'min' = 0,~~
    ~~'max' = 3000,~~
    ~~'palette' = ['red', 'green']~~
~~}~~

---
>title: "GEE画像表示におけるピクセル値とmin/max調整の基礎"
description: "Landsat画像のピクセル値の意味、min/maxの設定とクリッピングの関係、98パーセンタイルによる最適化手法を解説。"
tags: ["GEE", "ピクセル値", "リモートセンシング", "min/max", "可視化", "Landsat", "Obsidian"]
date: 2025-11-20
author: "Seiichi"
category: "GEE基礎解説"
---

>## ピクセル値とは何か？

>GEEで扱うリモートセンシング画像（例：Landsat）では、各ピクセルは「特定の波長の光をどれだけ反射したか」を数値で表しています。

>| ピクセル値の範囲 | 反射率の意味 | 表示上の見え方 | 主な地表例 |
>|------------------|--------------|----------------|-------------|
>| 0〜1000          | 非常に暗い   | 黒〜濃い青     | 水域（琵琶湖）、濃い影、焼け跡 |
>| 1000〜3000       | やや暗い     | 暗い緑〜灰色   | 森林、湿地、アスファルト |
>| 3000〜6000       | 中程度       | 緑〜黄          | 草地、農地、裸地 |
>| 6000〜9000       | 明るい       | 黄〜白          | 砂地、都市、雲 |
>| 9000〜10000      |非常に明るい  | 白              | 雲の端、雪氷、センサ飽和 |

---

>## なぜ max 値が重要なのか？

>- `vis_params` の `max` 値が低すぎると、明るい地表（例：砂浜、コンクリート、雲）がすべて白く表示されてしまい、**クリッピング（飽和）**が発生します。
>- 高すぎると、全体が暗くなり、**コントラストが失われます**。
>- 適切な `max` 値を設定するには、画像の統計的な明るさ分布を知る必要があります。

---

>## 最適な max 値を求める方法（98パーセンタイル）

```python
# 可視化対象のバンドを選択（例：自然色）
bands = ['SR_B4', 'SR_B3', 'SR_B2']

# 98パーセンタイルを計算
stats = best_image.select(bands).reduceRegion(
    reducer=ee.Reducer.percentile([98]),
    geometry=best_image.geometry(),
    scale=30,
    maxPixels=1e9
)

# SR_B4（赤バンド）の98パーセンタイル値を取得
max_value_98 = stats.get('SR_B4').getInfo()
print(f"推奨される max 値: {max_value_98}")
```

> &rarr;98パーセンタイルを使う理由：極端な値（雲やノイズ）を除外し、画像の大部分に適した明るさを得るため。

#### **$\text{NDVI}$ 表示パラメータの修正**

1. **Pythonの構文とバンド名の修正**
- **構文**: Pythonの辞書（{}）では、キーと値を区切るのに = ではなく : (コロン) を使います。
- **バンド名**: $\text{SR\_B4}$ と $\text{SR\_B5}$ は計算の入力として使いました。表示するのは、計算結果である新しいバンド名 'NDVI' です。

2. **$\text{Min}$ / $\text{Max}$ の修正**
$\text{NDVI}$ の値は常に -1.0 から +1.0 の範囲に収まる指数であり、元のLandsatのピクセル値（0〜10000）とはスケールが異なります。

|指標     |意味                    |
|--------|------------------------|
|-1.0    |水域（ほとんど光を反射しない |
|0       |岩、砂、人工物             |
|+1.0    |健康で密度の高い植生        |

これを踏まえ、以下の通り修正しましょう。

---
```python
ndvi_vis_params = {
    'bands': ['NDVI'],                  # 修正点1: 'NDVI'バンドを選択
    'min': -0.2,                        # 修正点2: 最小値を-1.0から-0.2付近に調整
    'max': 0.8,                         # 修正点3: 最大値を+1.0から0.8付近に調整
    # 水 (-1) から不健康な植生 (0) を経て、健康な植生 (+1) へと変化するパレットを設定
    'palette': ['blue', 'white', 'yellow', 'green', 'darkgreen']
}
```
---

- （補足: $\text{min}$ を-0.2に調整したのは、水域（-1.0）が多すぎると植生部分のコントラストが弱くなるのを防ぐためです）

## ステップ10: $\text{NDVI}$ の地図表示

これで、最終的な $\text{NDVI}$ 画像 (ndvi_image) と、適切な視覚化パラメータ (ndvi_vis_params) が揃いました。

#### **Guiding Question**: 最終ステップとして、以前使った表示ロジックを再利用し、ndvi_image と ndvi_vis_params を使ってタイルURLを取得し、地図に追加するにはどうすればよいでしょうか？

```python
map_id_dict = ndvi_image.getMapId(ndvi_vis_params)
tile_url = map_id_dict['tile_fetcher'].url_format
```

## ステップ11: $\text{NDVI}$ レイヤーの地図への追加

#### **Guiding Question:** 取得した tile_url を使って、Foliumの folium.TileLayer(...) を完成させ、それを my_map に追加するには、どのようにコードを記述すればよいでしょうか？レイヤーの名前は 'NDVI Composite' としてください。

```python
folium.TileLayer(
    tiles=tile_url,
    attr='Google Earth Engine',
    name= 'NDVI Composite',
    overlay=True,
    control=True
).add_to(my_map)
```

#### **完成コード**

In [ ]:
import ee
import datetime
import folium

In [ ]:
ee.Initialize(project='earth-change-analysis')

In [ ]:
biwako_point = ee.Geometry.Point([136.17, 35.10])
longitude, latitude = biwako_point.coordinates().getInfo() # 地図の中心座標を取得

In [ ]:
end_date = '2025-11-17' 
start_date = (datetime.datetime.strptime(end_date, '%Y-%m-%d') - datetime.timedelta(days=365)).strftime('%Y-%m-%d')

In [ ]:
l9_collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')

In [ ]:
filtered_collection = l9_collection \
    .filterDate(start_date, end_date) \
    .filterBounds(biwako_point)

In [ ]:
def mask_clouds(image):
    # 1. QA_PIXELバンドを選択
    qa = image.select('QA_PIXEL')
    
    # 2. 雲の重みを定義
    # Bit 3 (Cloud) = 8, Bit 1 (Dilated Cloud) = 2
    
    # 3. 雲・希釈雲の両方ではないピクセルをTrueとするマスクを作成
    mask = qa.bitwiseAnd(8).eq(0).And(  
             qa.bitwiseAnd(2).eq(0))   
           
    # 4. マスクを画像に適用
    return image.updateMask(mask)

In [ ]:
# 雲除去関数をコレクション内のすべての画像に適用
cloud_free_collection = filtered_collection.map(mask_clouds)

# 雲除去されたコレクション全体の中央値を計算し、1枚の合成画像を生成
final_composite = cloud_free_collection.median()

In [ ]:
def add_ndvi(image):
    nir = image.select('SR_B5')
    red = image.select('SR_B4')
    
    #  .rename('NDVI') を追加
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI') 
    
    return image.addBands(ndvi)

In [ ]:
ndvi_image = add_ndvi(final_composite)

In [ ]:
ndvi_vis_params = {
    'bands': ['NDVI'],                  # 修正点1: 'NDVI'バンドを選択
    'min': -0.2,               
    # 修正点2: 最小値を-1.0から-0.2付近に調整
    'max': 0.8,                         # 修正点3: 最大値を+1.0から0.8付近に調整
    # 水 (-1) から不健康な植生 (0) を経て、健康な植生 (+1) へと変化するパレットを設定
    'palette': ['blue', 'white', 'yellow', 'green', 'darkgreen']
}

In [ ]:
map_id_dict = ndvi_image.getMapId(ndvi_vis_params)
tile_url = map_id_dict['tile_fetcher'].url_format

In [ ]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [ ]:
my_map = folium.Map(
        location=[latitude, longitude],   # 地図の中心座標
        zoom_start=10,                    # 初期ズームレベル (琵琶湖周辺)
        tiles=basemaps['Google Satellite Hybrid'] # 初期タイルを設定
    )

In [ ]:
folium.TileLayer(
    tiles=tile_url,
    attr='Google Earth Engine',
    name= 'NDVI Composite',
    overlay=True,
    control=True
).add_to(my_map)

In [ ]:
# 6. レイヤーコントロールの追加
folium.LayerControl().add_to(my_map)

In [ ]:
# 7. マップの表示
display(my_map)